In [1]:
# Set up environment
import pandas as pd
import numpy as np
import json
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import re
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch
from transformers import BertForSequenceClassification, BertConfig
from transformers import BertPreTrainedModel, BertModel
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
from transformers import BertTokenizer
from IPython.display import clear_output
import time
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import torch.optim as optim
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.nn import CrossEntropyLoss, Dropout, Linear



In [2]:
PRETRAINED_MODEL_NAME = "bert-base-chinese"

# get tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

clear_output()
print("PyTorch version：", torch.__version__)

PyTorch version： 2.1.0+cu121


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Cleaning

In [ ]:
file_path = '12345_raw_data.xlsx'
data = pd.read_excel(file_path)

# extract the features and labels
classification_data = data[['来电内容', '举报专项']]

# show first rows
print(classification_data.head())

                                                来电内容    举报专项
0  市民于2023-12-17 ，反映违法建房占道种树和村民口粮地转租钱没发问题，见热线-231...  治理违法建设
1  市民反映，朝阳区网信大厦B座和霄云路辅路交叉口的地方，有卖麻辣烫的，还有个老奶奶卖烤肠的没有...  占道经营整治
2  市民反映，在房山区城关街道吉兴苑1号楼10单元3楼302是群租房，里面居住了很多人，非常的影...   群租房治理
3  工单来源：微信服务号，此工单内容为网民留言时填写的标题及原文。 \n============...  治理违法建设
4  市民反映，位于通州区宋庄镇北京工商大学嘉华学院南门存在街头游商现象，市民希望能够帮助得到解决...  占道经营整治


In [ ]:
# save as json
json_file_path = 'data_to_json.json'
classification_data.to_json(json_file_path, orient='records', force_ascii=False)

In [ ]:
with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)
print(df.head())

                                                来电内容    举报专项
0  市民于2023-12-17 ，反映违法建房占道种树和村民口粮地转租钱没发问题，见热线-231...  治理违法建设
1  市民反映，朝阳区网信大厦B座和霄云路辅路交叉口的地方，有卖麻辣烫的，还有个老奶奶卖烤肠的没有...  占道经营整治
2  市民反映，在房山区城关街道吉兴苑1号楼10单元3楼302是群租房，里面居住了很多人，非常的影...   群租房治理
3  工单来源：微信服务号，此工单内容为网民留言时填写的标题及原文。 \n============...  治理违法建设
4  市民反映，位于通州区宋庄镇北京工商大学嘉华学院南门存在街头游商现象，市民希望能够帮助得到解决...  占道经营整治


In [ ]:
df.info

<bound method DataFrame.info of                                                      来电内容      举报专项
0       市民于2023-12-17 ，反映违法建房占道种树和村民口粮地转租钱没发问题，见热线-231...    治理违法建设
1       市民反映，朝阳区网信大厦B座和霄云路辅路交叉口的地方，有卖麻辣烫的，还有个老奶奶卖烤肠的没有...    占道经营整治
2       市民反映，在房山区城关街道吉兴苑1号楼10单元3楼302是群租房，里面居住了很多人，非常的影...     群租房治理
3       工单来源：微信服务号，此工单内容为网民留言时填写的标题及原文。 \n============...    治理违法建设
4       市民反映，位于通州区宋庄镇北京工商大学嘉华学院南门存在街头游商现象，市民希望能够帮助得到解决...    占道经营整治
...                                                   ...       ...
255204  市民反映，在海淀区上庄镇海淀嘉俊小区与三嘉信苑小区之间红绿灯垃圾桶处有一辆白色轿车正在售卖水...    占道经营整治
255205  市民反映，在丰台区太平桥街道万泉盛景园7号楼3单元704，房间内有打隔断，里面居住了四户人。...     群租房治理
255206  市民反映，自家住在海淀区田村路街道阜石路兰德华庭2号楼1单元，市民称小区新换了物业，小区内没...  地下空间清理整治
255207  市民反映，2022年12月23日10:40海淀区海淀医院进口处旁边有夫妻两个人烤红薯和卖板栗...    占道经营整治
255208  市民反映，大兴区旧宫云龙家园21栋1单元602室，存在群租房，已经反映过没有处理，希望尽快给...     群租房治理

[255209 rows x 2 columns]>

In [ ]:
df.sample(10)

,来电内容,举报专项
12539,市民反映，之前于2023-11-29 反映小区地库出入口只开放一个问题，见热线-231129...,地下空间清理整治
199014,工单来源：微信服务号，此工单内容为网民留言时填写的标题及原文。 \n============...,治理违法建设
9908,市民反映，顺义区南彩镇九王庄村，村内有个村民（不清楚名字）住在48号，在公共区域搭建违建，并...,治理违法建设
197336,市民反映，平谷区金海湖镇祖务村有外村人在购买土地建墙圈占，希望帮助处理，来电反映圈占土地问题...,治理违法建设
122327,市民反映，东城区南颂年胡同5号楼3单元501，是2室1厅和阁楼，客厅被打成隔断，一共居住7个...,治理违法建设
48124,市民反映，在朝阳区丰和乡常营中路远洋3期1号，此地的围挡市民怀疑是超出范围了，怀疑有一些侵占...,施工围挡治理
102119,市民反映，延庆区（还没分属于哪个街乡镇）碧桂园小区2期40号楼111地下室，地下室严重漏水，...,地下空间清理整治
15793,市民反映，通州区张家湾镇里二泗村227号属于违建，期间自己反馈过此事，村委会回复自己称该房屋...,治理违法建设
29333,市民反映，于10月31日反映希望大队把合同剩余钱财退还给自己及违建问题。 见热线-23103...,治理违法建设
32889,工单来源：微信服务号，此工单内容为网民留言时填写的标题及原文。 \n============...,治理违法建设


In [ ]:
# notice some data has missing values, upon checking the original data, consider this as invalid data.
def clean_text(text):
    if text.startswith('工单来源：'):
        # remove "工单来源：" to "="
        text = re.sub(r"工单来源：.*?={2,}", "", text, flags=re.DOTALL)
        text = text.lstrip('\n').replace('\n留言', '')
        # check if end with“-”，if so，remove the content after “-”
        text = re.sub(r"-.*$", "", text, flags=re.DOTALL)
    text = re.sub(r"工单流转.*$", "", text, flags=re.DOTALL)
    return text

# apply
df['来电内容'] = df['来电内容'].apply(clean_text)

# display
df.head()

,来电内容,举报专项
0,市民于2023-12-17 ，反映违法建房占道种树和村民口粮地转租钱没发问题，见热线-231...,治理违法建设
1,市民反映，朝阳区网信大厦B座和霄云路辅路交叉口的地方，有卖麻辣烫的，还有个老奶奶卖烤肠的没有...,占道经营整治
2,市民反映，在房山区城关街道吉兴苑1号楼10单元3楼302是群租房，里面居住了很多人，非常的影...,群租房治理
3,\n留言标题：对12345回复有疑问，请人工联系本人，谢谢\n留言原文：2023.12.21...,治理违法建设
4,市民反映，位于通州区宋庄镇北京工商大学嘉华学院南门存在街头游商现象，市民希望能够帮助得到解决...,占道经营整治


In [ ]:
#  View null values
df.isna().sum()

来电内容    0
举报专项    0
dtype: int64

In [ ]:
df.describe()

,来电内容,举报专项
count,255209,255209
unique,245080,11
top,朝阳路和十里堡路丁字路口西北角，去年盖起一违章建筑，面积约130平米。室内装有空调，照明。要...,治理违法建设
freq,55,139736


In [ ]:
duplicates = df[df.duplicated('来电内容', keep=False)]

# sort to adjacent
duplicates = duplicates.sort_values('来电内容')

# display
print(duplicates)

                                                     来电内容    举报专项
210406  \n\n\n海淀区天阅西山社区存在几十处违法建设，这些违建与社区规划建设方案严重不符，但是规...  治理违法建设
210630  \n\n\n海淀区天阅西山社区存在几十处违法建设，这些违建与社区规划建设方案严重不符，但是规...  治理违法建设
210639  \n\n\n海淀区天阅西山社区存在几十处违法建设，这些违建与社区规划建设方案严重不符，但是规...  治理违法建设
210687  \n\n\n海淀区天阅西山社区存在几十处违法建设，这些违建与社区规划建设方案严重不符，但是规...  治理违法建设
244528  \n\n\n社区内部大量非法的建筑自收房之日起一直存在至今，群众多次投诉举报，相关部门就是一...  治理违法建设
...                                                   ...     ...
250177               静安里中街能仁居三村烧烤距是否有建筑规划许可，什么时候能提供产权相关资料  治理违法建设
250178               静安里中街能仁居三村烧烤距是否有建筑规划许可，什么时候能提供产权相关资料  治理违法建设
250179               静安里中街能仁居三村烧烤距是否有建筑规划许可，什么时候能提供产权相关资料  治理违法建设
253151  顺义区牛栏山史家口村赵瑞占到私搭乱建，堆积杂物，致使我多年无法通行，存在严重安全隐患。该问题...  治理违法建设
250756  顺义区牛栏山史家口村赵瑞占到私搭乱建，堆积杂物，致使我多年无法通行，存在严重安全隐患。该问题...  治理违法建设

[16070 rows x 2 columns]


In [ ]:
# check if the duplicated sample has the same label
duplicates_grouped = duplicates.groupby('来电内容')['举报专项'].nunique()
different_labels = duplicates_grouped[duplicates_grouped > 1].index
different_labels_data = duplicates[duplicates['来电内容'].isin(different_labels)]

print(different_labels_data)



                                                     来电内容      举报专项
558      \n留言标题：投诉紫竹院街道\n留言原文：在悦园地下室群租治理或地下室违规出租方面，紫竹院...  地下空间清理整治
5913     \n留言标题：投诉紫竹院街道\n留言原文：在悦园地下室群租治理或地下室违规出租方面，紫竹院...  地下空间清理整治
9488     \n留言标题：投诉紫竹院街道\n留言原文：在悦园地下室群租治理或地下室违规出租方面，紫竹院...  地下空间清理整治
14187    \n留言标题：投诉紫竹院街道\n留言原文：在悦园地下室群租治理或地下室违规出租方面，紫竹院...  地下空间清理整治
19279    \n留言标题：投诉紫竹院街道\n留言原文：在悦园地下室群租治理或地下室违规出租方面，紫竹院...     群租房治理
...                                                   ...       ...
252361  紫郡兰园小区13-2-102业主将地下仓储改造用于居住，违规破坏房屋主体结构且占用消防通道，...  地下空间清理整治
252711  紫郡兰园小区13-2-102业主将地下仓储改造用于居住，违规破坏房屋主体结构且占用消防通道，...    治理违法建设
252712  紫郡兰园小区13-2-102业主将地下仓储改造用于居住，违规破坏房屋主体结构且占用消防通道，...  地下空间清理整治
255123  \n海保发打着便民惠民的旗号非法开餐饮\n\n海淀区保障性住房发展有限公司身为区属重点国企，...    治理违法建设
255146  \n海保发打着便民惠民的旗号非法开餐饮\n\n海淀区保障性住房发展有限公司身为区属重点国企，...  无证无照经营整治

[309 rows x 2 columns]


In [ ]:
df = df[~df['来电内容'].isin(different_labels)]

In [ ]:
duplicates = df[df.duplicated('来电内容', keep=False)]

# sort to adjacent
duplicates = duplicates.sort_values('来电内容')

# display
print(duplicates)

                                                     来电内容    举报专项
210687  \n\n\n海淀区天阅西山社区存在几十处违法建设，这些违建与社区规划建设方案严重不符，但是规...  治理违法建设
210639  \n\n\n海淀区天阅西山社区存在几十处违法建设，这些违建与社区规划建设方案严重不符，但是规...  治理违法建设
210630  \n\n\n海淀区天阅西山社区存在几十处违法建设，这些违建与社区规划建设方案严重不符，但是规...  治理违法建设
210406  \n\n\n海淀区天阅西山社区存在几十处违法建设，这些违建与社区规划建设方案严重不符，但是规...  治理违法建设
244412  \n\n\n社区内部大量非法的建筑自收房之日起一直存在至今，群众多次投诉举报，相关部门就是一...  治理违法建设
...                                                   ...     ...
250179               静安里中街能仁居三村烧烤距是否有建筑规划许可，什么时候能提供产权相关资料  治理违法建设
250178               静安里中街能仁居三村烧烤距是否有建筑规划许可，什么时候能提供产权相关资料  治理违法建设
250177               静安里中街能仁居三村烧烤距是否有建筑规划许可，什么时候能提供产权相关资料  治理违法建设
250756  顺义区牛栏山史家口村赵瑞占到私搭乱建，堆积杂物，致使我多年无法通行，存在严重安全隐患。该问题...  治理违法建设
253151  顺义区牛栏山史家口村赵瑞占到私搭乱建，堆积杂物，致使我多年无法通行，存在严重安全隐患。该问题...  治理违法建设

[15761 rows x 2 columns]


In [ ]:
# Drop the duplications
df = df.drop_duplicates()

In [ ]:
df

,来电内容,举报专项
0,市民于2023-12-17 ，反映违法建房占道种树和村民口粮地转租钱没发问题，见热线-231...,治理违法建设
1,市民反映，朝阳区网信大厦B座和霄云路辅路交叉口的地方，有卖麻辣烫的，还有个老奶奶卖烤肠的没有...,占道经营整治
2,市民反映，在房山区城关街道吉兴苑1号楼10单元3楼302是群租房，里面居住了很多人，非常的影...,群租房治理
3,\n留言标题：对12345回复有疑问，请人工联系本人，谢谢\n留言原文：2023.12.21...,治理违法建设
4,市民反映，位于通州区宋庄镇北京工商大学嘉华学院南门存在街头游商现象，市民希望能够帮助得到解决...,占道经营整治
...,...,...
255204,市民反映，在海淀区上庄镇海淀嘉俊小区与三嘉信苑小区之间红绿灯垃圾桶处有一辆白色轿车正在售卖水...,占道经营整治
255205,市民反映，在丰台区太平桥街道万泉盛景园7号楼3单元704，房间内有打隔断，里面居住了四户人。...,群租房治理
255206,市民反映，自家住在海淀区田村路街道阜石路兰德华庭2号楼1单元，市民称小区新换了物业，小区内没...,地下空间清理整治
255207,市民反映，2022年12月23日10:40海淀区海淀医院进口处旁边有夫妻两个人烤红薯和卖板栗...,占道经营整治


In [ ]:
df = df.reset_index(drop=True)

In [ ]:
data_dict = df.to_dict(orient='records')

json_file_name = 'data_cleaned.json'

with open(json_file_name, 'w') as json_file:
    json.dump(data_dict, json_file)

# Tokenization

In [15]:
# json_file_name = '/content/drive/MyDrive/reporyt-predict/data_cleaned.json'
with open(json_file_name, 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)
print(df.head())

                                                来电内容    举报专项
0  市民于2023-12-17 ，反映违法建房占道种树和村民口粮地转租钱没发问题，见热线-231...  治理违法建设
1  市民反映，朝阳区网信大厦B座和霄云路辅路交叉口的地方，有卖麻辣烫的，还有个老奶奶卖烤肠的没有...  占道经营整治
2  市民反映，在房山区城关街道吉兴苑1号楼10单元3楼302是群租房，里面居住了很多人，非常的影...   群租房治理
3  \n留言标题：对12345回复有疑问，请人工联系本人，谢谢\n留言原文：2023.12.21...  治理违法建设
4  市民反映，位于通州区宋庄镇北京工商大学嘉华学院南门存在街头游商现象，市民希望能够帮助得到解决...  占道经营整治


In [16]:
df['举报专项'].unique()

array(['治理违法建设', '占道经营整治', '群租房治理', '无证无照经营整治', '地下空间清理整治',
       '区域性市场（区域性市场疏解提升、商圈商场改造）', '施工围挡治理', '发展老年餐桌', '建设提升基本便民服务网点',
       '"开墙打洞"整治', '桥下空间治理提升'], dtype=object)

In [4]:
bert_chinese_tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [5]:
from sklearn.model_selection import train_test_split
import random

class ComplaintsDataset(Dataset):
    def __init__(self, mode, tokenizer, json_file_name, split_ratio=0.8, sample_ratio=0.01):
        assert mode in ["train", "val"]
        self.mode = mode
        with open(json_file_name, 'r', encoding='utf-8') as f:
            data = json.load(f)
        df = pd.DataFrame(data)

        # Sample a portion of the data
        df_sampled = df.sample(frac=sample_ratio, random_state=42)

        # Split the data set into a training set and a validation set
        train_df, val_df = train_test_split(df_sampled, train_size=split_ratio, random_state=42)
        self.df = train_df if mode == "train" else val_df
        self.len = len(self.df)

        # Create a label map
        self.label_map = {label: i for i, label in enumerate(df['举报专项'].unique())}
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        text = self.df.iloc[idx]['来电内容']
        label = self.df.iloc[idx]['举报专项']
        label_id = self.label_map[label]
        label_tensor = torch.tensor(label_id)

        # split tokens
        word_pieces = ["[CLS]"]
        tokens = self.tokenizer.tokenize(text)
        word_pieces += tokens + ["[SEP]"]
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)

        segments_tensor = torch.tensor([0] * len(word_pieces), dtype=torch.long)

        return (tokens_tensor, segments_tensor, label_tensor)

    def __len__(self):
        return self.len



In [6]:
# json_file_name = 'data_cleaned.json'

# Colab
json_file_name = '/content/drive/MyDrive/reporyt-predict/data_cleaned.json'
trainset = ComplaintsDataset("train", tokenizer, json_file_name, split_ratio=0.8, sample_ratio=1)
valset = ComplaintsDataset("val", tokenizer, json_file_name, split_ratio=0.8, sample_ratio=1)



In [ ]:
# Select the first sample
sample_idx = 0

# Extract the original text from the dataset and compare it to the label
text, label = trainset.df.iloc[sample_idx][['来电内容', '举报专项']]

# Take out the transformed ID tensors from the Dataset you just created
tokens_tensor, segments_tensor, label_tensor = trainset[sample_idx]

# Restore tokens to text
tokens = bert_chinese_tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
combined_text = "".join(tokens)

# Before and after rendering
print(f"""[原始文本]
内容：{text}
分类：{label}

--------------------

[Dataset 返回的 tensors]
tokens_tensor  : {tokens_tensor}

segments_tensor : {segments_tensor}

label_tensor   : {label_tensor}

--------------------

[还原 tokens_tensors]
{combined_text}
""")


[原始文本]
内容：市民反映，海淀区西北旺镇永丰屯村手机店都是无照经营，市民表示已经很长时间了，市民之前没有联系过其他部门，市场监管局也只是关了没有营业执照的饭店，其他店铺没有关闭，来电反映店铺无照经营问题
注：市民要求保密个人信息（已解释）。
分类：无证无照经营整治

--------------------

[Dataset 返回的 tensors]
tokens_tensor  ：tensor([ 101, 2356, 3696, 1353, 3216, 8024, 3862, 3895, 1277, 6205, 1266, 3200,
        7252, 3719,  705, 2254, 3333, 2797, 3322, 2421, 6963, 3221, 3187, 4212,
        5307, 5852, 8024, 2356, 3696, 6134, 4850, 2347, 5307, 2523, 7270, 3198,
        7313,  749, 8024, 2356, 3696,  722, 1184, 3766, 3300, 5468, 5143, 6814,
        1071,  800, 6956, 7305, 8024, 2356, 1767, 4664, 5052, 2229,  738, 1372,
        3221, 1068,  749, 3766, 3300, 5852,  689, 2809, 4212, 4638, 7649, 2421,
        8024, 1071,  800, 2421, 7215, 3766, 3300, 1068, 7308, 8024, 3341, 4510,
        1353, 3216, 2421, 7215, 3187, 4212, 5307, 5852, 7309, 7579, 3800, 8038,
        2356, 3696, 6206, 3724,  924, 2166,  702,  782,  928, 2622, 8020, 2347,
        6237, 7025, 8021,  511,  102])

segments_tensor：tensor([0, 0, 0, 0, 0, 0, 0,

In [7]:
# 这个函数的输入 `samples` 是一个列表，其中的每个元素都是
# `ComplaintsDataset` 返回的一个样本，每个样本都包含 3 个张量：
# - tokens_tensor
# - segments_tensor
# - label_tensor
# 它将对 tokens_tensor 进行 zero padding，并生成 masks_tensors

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler


def create_mini_batch(samples, max_seq_length=512):
    tokens_tensors = [s[0] for s in samples]
    masks_tensors = [s[1] for s in samples]

    # cut down overlength series
    tokens_tensors = [t if t.size(0) <= max_seq_length else t[:max_seq_length] for t in tokens_tensors]
    masks_tensors = [m if m.size(0) <= max_seq_length else m[:max_seq_length] for m in masks_tensors]

    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None

    tokens_tensors = pad_sequence(tokens_tensors, batch_first=True)
    masks_tensors = pad_sequence(masks_tensors, batch_first=True)
    masks_tensors = masks_tensors.masked_fill(tokens_tensors != 0, 1)

    return tokens_tensors, masks_tensors, label_ids

# set batch size
BATCH_SIZE = 32

#
trainloader = DataLoader(trainset, sampler = RandomSampler(trainset), batch_size=BATCH_SIZE, collate_fn=lambda x: create_mini_batch(x, max_seq_length=512))
valloader = DataLoader(valset, sampler = SequentialSampler(valset), batch_size=BATCH_SIZE, collate_fn=lambda x: create_mini_batch(x, max_seq_length=512))

In [ ]:
len(trainloader)

6126

In [ ]:
data = next(iter(trainloader))

tokens_tensors, masks_tensors, label_ids = data

print(f"""
tokens_tensors.shape   = {tokens_tensors.shape}
{tokens_tensors}
------------------------
masks_tensors.shape    = {masks_tensors.shape}
{masks_tensors}
------------------------
label_ids.shape        = {label_ids.shape}
{label_ids}
""")



tokens_tensors.shape   = torch.Size([32, 512])
tensor([[ 101, 2356, 3696,  ...,    0,    0,    0],
        [ 101, 2356, 3696,  ...,    0,    0,    0],
        [ 101, 2356, 3696,  ...,    0,    0,    0],
        ...,
        [ 101, 2356, 3696,  ...,    0,    0,    0],
        [ 101, 4522, 6241,  ...,    0,    0,    0],
        [ 101, 3255, 2110,  ...,    0,    0,    0]])
------------------------
masks_tensors.shape    = torch.Size([32, 512])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
------------------------
label_ids.shape        = torch.Size([32])
tensor([0, 2, 9, 6, 3, 1, 1, 0, 0, 1, 0, 0, 3, 0, 0, 1, 3, 3, 0, 2, 0, 0, 4, 0,
        0, 0, 0, 2, 0, 1, 0, 4])



# Bert Training

In [8]:
PRETRAINED_MODEL_NAME = "bert-base-chinese"
NUM_LABELS = len(trainset.label_map)

# load pretrained model and configure
config = BertConfig.from_pretrained(PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)
model = BertForSequenceClassification.from_pretrained(PRETRAINED_MODEL_NAME, config=config)

# display the module in model
print("name            module\n----------------------")
for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


name            module
----------------------
bert:embeddings
bert:encoder
bert:pooler
dropout         Dropout(p=0.1, inplace=False)
classifier      Linear(in_features=768, out_features=11, bias=True)


In [9]:
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0

    with torch.no_grad():
        for data in dataloader:
            # transfer tensors to GPU
            if next(model.parameters()).is_cuda:
                data = [t.to(device) for t in data if t is not None]

            tokens_tensors, masks_tensors, labels = data

            outputs = model(input_ids=tokens_tensors, attention_mask=masks_tensors)
            logits = outputs.logits
            _, pred = torch.max(logits.data, 1)

            # compute the accuracy
            if compute_acc:
                total += labels.size(0)
                correct += (pred == labels).sum().item()

            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))

    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions


In [10]:
# send model to device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# get the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# pretrained large model, so only takes tiny epochs
EPOCHS = 2
for epoch in range(EPOCHS):
    running_loss = 0.0
    correct = 0
    total = 0
    model.train()

    progress_bar = tqdm(enumerate(trainloader), total=len(trainloader), desc=f"Epoch {epoch + 1}/{EPOCHS}")
    for i, data in progress_bar:
        tokens_tensors, masks_tensors, labels = [t.to(device) for t in data]

        optimizer.zero_grad()

        outputs = model(input_ids=tokens_tensors, attention_mask=masks_tensors, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.logits, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        train_acc = correct / total

        # output train result per 40 batchs
        if (i + 1) % 40 == 0 or i == len(trainloader) - 1:
            progress_bar.set_postfix({
                'Batch': f"{i + 1}/{len(trainloader)}",
                'Loss': f"{running_loss/(i+1):.4f}",
                'Train Acc': f"{train_acc:.4f}"
            })

    model.eval()
    _, val_acc = get_predictions(model, valloader, compute_acc=True)

    # output the acc of the this epoch
    print(f"\nEnd of epoch {epoch + 1}, Train Loss: {running_loss/len(trainloader):.4f}, "
          f"Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

Epoch 1/2: 100%|██████████| 6126/6126 [1:07:41<00:00,  1.51it/s, Batch=6126/6126, Loss=0.1074, Train Acc=0.9702]



End of epoch 1, Train Loss: 0.1074, Train Acc: 0.9702, Val Acc: 0.9772


Epoch 2/2: 100%|██████████| 6126/6126 [1:07:25<00:00,  1.51it/s, Batch=6126/6126, Loss=0.0614, Train Acc=0.9811]



End of epoch 2, Train Loss: 0.0614, Train Acc: 0.9811, Val Acc: 0.9788


# Save Model

In [ ]:
from google.colab import files

# save model
model_save_path = 'bert_finetuned_model.pth'
torch.save(model, model_save_path)

# download
files.download(model_save_path)

In [17]:
label_map = {label: i for i, label in enumerate(df['举报专项'].unique())}

# save label_map
with open('label_map.json', 'w', encoding='utf-8') as f:
    json.dump(label_map, f, ensure_ascii=False)

In [18]:
from google.colab import files

# save model
model_save_path = 'bert_finetuned_model_1.pth'
torch.save(model, model_save_path)